## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

In [10]:
def exe_df(stmt:str,engine:sqlalchemy.engine=engine)->pd.DataFrame:
    """
    sql(select)を実行してdataframeを返す
    """
    with engine.connect() as conn:
        df=pd.read_sql(stmt,conn)
    return df 

## List

In [11]:
ct("List",6,1)
exe("""
select sum(出金額)
from 家計簿
""")

[(15740,)]

In [12]:
ct("List",6,2)
exe("""
select sum(出金額),
avg(出金額)
from 家計簿
""")

[(15740, Decimal('3148.0000000000000000'))]

In [13]:
ct("List",6,3)
exe("""
select count(*) from 家計簿 where 費目='食費'
""")

[(1,)]

In [14]:
ct("List",6,5)
exe("""
select avg(coalesce(出金額,0))
from 家計簿
""")

[(Decimal('3148.0000000000000000'),)]

In [15]:
ct("List",6,7)
exe("""
select 費目,sum(出金額)
from 家計簿
group by 費目
""")

[('食費', 380), ('給料', 0), ('交際費', 5000), ('教養娯楽費', 2800), ('水道光熱費', 7560)]

In [16]:
ct("List",6,9)
exe("""
select 費目,sum(出金額)
from 家計簿
group by 費目
having sum(出金額)>0
""")

[('食費', 380), ('交際費', 5000), ('教養娯楽費', 2800), ('水道光熱費', 7560)]

In [17]:
ct("List",6,10)
exe("""
select sum(入金額),sum(出金額)
from 口座入出金テーブル
where 日付>='2017-01-01' and 日付<'2018-01-01'
""")

[(280000, 146140)]

## Q

In [18]:
ct("Q",6,1,1)
exe("""
select sum(降水量),avg(最高気温),avg(最低気温)
from 都市別気象観測
""")

[(4312, Decimal('25.6944444444444444'), Decimal('6.0000000000000000'))]

In [19]:
ct("Q",6,1,3)
exe("""
select 都市名,avg(降水量),min(最高気温),max(最低気温)
from 都市別気象観測
group by 都市名
""")

[('東京', Decimal('127.0000000000000000'), 11, 22),
 ('博多', Decimal('137.2500000000000000'), 14, 21),
 ('熊谷', Decimal('91.9166666666666667'), 12, 21),
 ('奈良', Decimal('130.1666666666666667'), 11, 19)]

In [20]:
ct("Q",6,1,5)
exe("""
select 都市名,max(最高気温)
from 都市別気象観測
group by 都市名
having max(最高気温)>=38
""")

[('熊谷', 38)]

In [22]:
ct("Q",6,2,1)
exe("""
select count(*)
from 入退室管理
where 退室 is null
""")

[(2,)]

In [23]:
ct("Q",6,2,2)
exe("""
select 社員名,count(*)
from 入退室管理
group by 社員名
order by 2 desc
""")

[('菅原', 3), ('湊', 2), ('朝香', 1)]

In [24]:
ct("Q",6,2,3)
exe("""
select 
case 事由区分
when '1' then 'メンテナンス'
when '2' then 'リリース作業'
else 'その他'
end
,count(*)
from 入退室管理
group by 事由区分
""")

[('リリース作業', 1), ('その他', 1), ('その他', 2), ('メンテナンス', 2)]

In [25]:
ct("Q",6,2,4)
exe("""
select 社員名,count(*)
from 入退室管理
group by 社員名
having count(*)>10
""")

[('菅原', 12)]

In [26]:
ct("Q",6,2,5)
exe("""
select 日付,count(社員名)
from 入退室管理
where 事由区分='3'
group by 日付
""")

[(datetime.date(2018, 2, 5), 2),
 (datetime.date(2018, 2, 13), 3),
 (datetime.date(2018, 2, 17), 1),
 (datetime.date(2018, 2, 18), 1)]